In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [13]:
#core_profiles = FUSE.core_profiles();
#resize!(core_profiles.profiles_1d,1)
#profiles_1d = core_profiles.profiles_1d[1]

profiles_1d = FUSE.core_profiles__profiles_1d()

ne0 = 1E20
Te0 = 1E3

@time begin
profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=21)
profiles_1d.electrons.density = ne0.*(1.0 .- profiles_1d.grid.rho_tor_norm.^2);
profiles_1d.electrons.temperature = (x;_...) -> Te0.*(1.0 .- x.^2);

# println(profiles_1d.electrons.density(0.0))
# profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=5);
# (profiles_1d.electrons.pressure);
# profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=21)
println(profiles_1d.electrons.pressure)
end;

[16021.8, 15941.79113625, 15702.96618, 15308.93003625, 14765.69088, 14081.66015625, 13267.65258, 12336.886136250001, 11304.98208, 10189.96493625, 9012.262499999999, 7794.70583625, 6562.52928, 5343.370436249998, 4167.27018, 3066.6726562500003, 2076.4252799999986, 1233.7787362500007, 578.3869799999997, 152.3072362500001, 0.0]
  0.358408 seconds (909.10 k allocations: 73.121 MiB, 2.30% gc time, 97.42% compilation time)


In [36]:
function fds_ancestors(fds)::Dict{Symbol,Union{Missing,FDS}}
    # ancestors will hold pointers to the ancestors in the fds struct
    ancestors = Dict()
    # initialize ancestors to missing
    path = f2p(fds)
    for p in path
        if typeof(p) <: String
            ancestors[Symbol(p)] = missing
        end
    end
    # traverse ancestors and assign pointers
    h = fds
    while h !== missing
        path = f2p(h)
        if typeof(path[end]) <: String
            ancestors[Symbol(path[end])] = h
        elseif path[end]!=0
            ancestors[Symbol(path[end-1])] = h
        end
        h = h._parent.value
    end
    return ancestors
end

"""
    exec_function_in_fds_namespace(fds::FDS, func::Function, func_args)

Execute a function passing the FDS stack as arguments to the function

# Arguments
- `fds::FDS`: FDS structure
- `func::Function`: function to be executed should accept in inputs the list of symbols that are traversed to get to the `fds`
- `func_args`: arguments passed to func, in addition to the list of symbols that are traversed to get to the `fds`

# Example `func`

    function pressure(x; core_profiles, electrons, profiles_1d)
        return electrons.temperature.*electrons.density * 1.60218e-19
    end

# Other example of valid `func`, now using argument splatting

    function pressure(x; electrons, others...)
        return electrons.temperature.*electrons.density * 1.60218e-19
    end
"""
function exec_function_with_fds_ancestors(fds, func, func_args)
    ancestors = fds_ancestors(fds)
    # execute function with ancestors arguments
    func(func_args...;ancestors...)
end
            
function pressure(x; core_profiles, electrons, profiles_1d)
    return electrons.temperature.*electrons.density * 1.60218e-19
end

exec_function_with_fds_ancestors(profiles_1d.electrons,pressure,1)

21-element Vector{Float64}:
 16021.8
 15941.79113625
 15702.96618
 15308.93003625
 14765.69088
 14081.66015625
 13267.65258
 12336.886136250001
 11304.98208
 10189.96493625
  9012.262499999999
  7794.70583625
  6562.52928
  5343.370436249998
  4167.27018
  3066.6726562500003
  2076.4252799999986
  1233.7787362500007
   578.3869799999997
   152.3072362500001
     0.0

In [10]:
#profiles_1d.electrons.density = x -> plasmarc[:ne0]*(1.0 .- x.^2).^plasmarc[:Sn]


# opaque closures


profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=11)
println(profiles_1d.electrons.pressure)

[16021.8, 15941.79113625, 15702.96618, 15308.93003625, 14765.69088, 14081.66015625, 13267.65258, 12336.886136250001, 11304.98208, 10189.96493625, 9012.262499999999, 7794.70583625, 6562.52928, 5343.370436249998, 4167.27018, 3066.6726562500003, 2076.4252799999986, 1233.7787362500007, 578.3869799999997, 152.3072362500001, 0.0]
[16021.8, 15702.96618, 14765.69088, 13267.65258, 11304.98208, 9012.262499999999, 6562.52928, 4167.27018, 2076.4252799999986, 578.3869799999997, 0.0]


In [71]:
data = dd();
resize!(data.core_profiles.profiles_1d,3)
data.core_profiles.profiles_1d[1].grid.rho_tor_norm = range(0,stop=1,length=10)
data.core_profiles.profiles_1d[2].grid.rho_tor_norm = range(0,stop=1,length=3)

coordinates(data.core_profiles.profiles_1d[1].electrons,:temperature)
coordinates(data.core_profiles.profiles_1d[2].electrons,:temperature)
coordinates(data.core_profiles.profiles_1d[3].electrons,:temperature)

Dict{Symbol, Vector{Any}} with 2 entries:
  :values => [missing]
  :names  => ["core_profiles.profiles_1d[:].grid.rho_tor_norm"]

In [61]:
fds=data.core_profiles.profiles_1d[2].electrons
coord="core_profiles.profiles_1d[:].grid.rho_tor_norm"

fds1 = f2fs(fds)
fds2 = u2fs(p2i(i2p(coord)[1:end-1]))
cs,s1,s2=FUSE.common_base_string(fds1,fds2)
common_fds = replace(string(cs),r"___$"=>"")

# go upstream until common acestor
h = fds
while f2fs(h)!=common_fds
    h=h._parent.value
end
for k in i2p(s2)
    h=getproperty(h,Symbol(k))
end
coord_leaf=i2p(coord)[end]
getproperty(h,Symbol(coord_leaf))

In [17]:
data = dd();
resize!(data.core_profiles.profiles_1d,3)
data.core_profiles.profiles_1d[1].grid.rho_tor_norm = range(0.0,stop=1.0,length=10)
data.core_profiles.profiles_1d[2].grid.rho_tor_norm = range(0.0,stop=1.0,length=3)

data.core_profiles.profiles_1d[1].electrons.temperature = 1.0.-(data.core_profiles.profiles_1d[1].grid.rho_tor_norm).^2
data.core_profiles.profiles_1d[2].electrons.temperature = x->1.0.-x.^2

data.core_profiles.profiles_1d[1].grid.rho_tor_norm = range(0.0,stop=1.0,length=3)
println(data.core_profiles.profiles_1d[1].electrons.temperature)
data.core_profiles.profiles_1d[2].grid.rho_tor_norm = range(0.0,stop=1.0,length=4)
println(data.core_profiles.profiles_1d[2].electrons.temperature)

data.core_profiles.profiles_1d[1].grid.rho_tor_norm = range(0,stop=1,length=100)
data.core_profiles.profiles_1d[2].grid.rho_tor_norm = range(0,stop=1,length=100)

data.core_profiles.profiles_1d[1].electrons.temperature

[1.0, 0.7469135802469136, 0.0]
[1.0, 0.8888888888888888, 0.5555555555555556, 0.0]


100-element FUSE.FDVector:
 1.0
 0.9988776655443322
 0.9977553310886644
 0.9966329966329966
 0.9955106621773288
 0.994388327721661
 0.9932659932659932
 0.9921436588103254
 0.9910213243546576
 0.9898989898989898
 0.988776655443322
 0.9876543209876543
 0.9842873176206508
 ⋮
 0.2098765432098766
 0.19079685746352404
 0.17171717171717174
 0.1526374859708192
 0.1335578002244669
 0.11447811447811436
 0.09539842873176202
 0.07631874298540972
 0.05723905723905718
 0.03815937149270485
 0.01907968574635233
 0.0

In [22]:
"""
    f2e(fds::Union{FDS,FDSvector};stop_at_top::Bool=true)::Vector{Union{String,Int}}

Returns IMAS location of a given FDS

Paremeter `stop_at_top` true returns path up to the top level FDS

NOTE: if `stop_at_top=false` then indexes of arrays of structures that cannot be determined are set to 0
"""
function f2e(fds::Union{FDS,FDSvector};stop_at_top::Bool=true)::Vector{Union{String,Int}}
    return f2e(fds, missing, nothing, Int[];stop_at_top=stop_at_top)
end

function f2e(fds::Union{FDS,FDSvector},
             child::Union{Missing,FDS,FDSvector},
             path::Union{Nothing,Vector},
             index::Vector{Int};
             stop_at_top::Bool=true)
    # initialize path and index
    if path === nothing
        if typeof(fds) <: FDS
            if typeof(fds._parent.value) <: FDSvector
                name = string(Base.typename(typeof(fds)).name) * "___"
            else
                name = string(Base.typename(typeof(fds)).name)
            end
        elseif typeof(fds) <: FDSvector
            name = string(Base.typename(eltype(fds)).name) * "___"
        end
        path = replace(name, "___" => "__:__")
        path = Vector{Any}(Vector{String}(split(path, "__")))
        path = [k == ":" ? 0 : k for k in path if length(k) > 0]
    end

    # collect integers for arrays of structures
    if typeof(fds) <: FDSvector
        ix = findfirst([k === child for k in fds.value])
        if ix === nothing
            push!(index, 0)
        else
            push!(index, ix)
        end
    end

    # traverse FDSs upstream or return result if got to the top
    if fds._parent.value === missing
        index = reverse(index)
        path = reverse([(typeof(k) <: Int) & (length(index)>0) ? pop!(index) : k for k in reverse(path)])
        return path
    else
        return f2e(fds._parent.value, fds, path, index; stop_at_top=stop_at_top)
    end
end

f2e(data.core_profiles.profiles_1d[1].electrons)

4-element Vector{Union{Int64, String}}:
  "core_profiles"
  "profiles_1d"
 1
  "electrons"

In [10]:
Pkg.test("FUSE")

     Testing FUSE
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_3TkYkg/Project.toml`
  [e64856f0] FUSE v0.1.0 `~/.julia/dev/FUSE`
  [a98d9a8b] Interpolations v0.13.4
  [682c06a0] JSON v0.21.2
  [c03570c3] Memoize v0.4.4
  [92933f4c] ProgressMeter v1.7.1
  [67601950] Quadrature v1.9.0
  [8dfed614] Test `@stdlib/Test`
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_3TkYkg/Manifest.toml`
  [621f4979] AbstractFFTs v1.0.1
  [79e6a3ab] Adapt v3.3.1
  [4fba245c] ArrayInterface v3.1.23
  [13072b0f] AxisAlgorithms v1.0.0
  [082447d4] ChainRules v0.8.24
  [d360d2e6] ChainRulesCore v0.10.13
  [861a8166] Combinatorics v1.0.2
  [38540f10] CommonSolve v0.2.0
  [bbf7d656] CommonSubexpressions v0.3.0
  [34da2185] Compat v3.32.0
  [187b0558] ConstructionBase v1.3.0
  [9a962f9c] DataAPI v1.7.0
  [864edb3b] DataStructures v0.18.10
  [e2d170a0] DataValueInterfaces v1.0.0
  [2b5f629d] DiffEqBase v6.70.0
  [163ba53b] DiffResults v1.0.3
  [b552c78f] DiffR

Test Summary: | Pass  Total
FDS           |   14     14
Test Summary: | Pass  Total
FDS_IMAS      |   15     15
JSON_IO: Error During Test at /Users/meneghini/.julia/dev/FUSE/test/runtests_fds.jl:90
  Got exception outside of a @test
  MethodError: Cannot `convert` an object of type Vector{Any} to an object of type FUSE.AbstractFDArray{Float64, 1}
  Closest candidates are:
    convert(::Type{T}, ::LinearAlgebra.Factorization) where T<:AbstractArray at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:58
    convert(::Type{T}, ::T) where T<:AbstractArray at abstractarray.jl:14
    convert(::Type{T}, ::T) where T at essentials.jl:205
  Stacktrace:
    [1] dict2fuse(dct::Dict{String, Any}, fds::FUSE.equilibrium; verbose::Bool, path::Vector{String})
      @ FUSE ~/.julia/dev/FUSE/src/data.jl:66
    [2] dict2fuse(dct::Dict{String, Any}, fds::FUSE.dd; verbose::Bool, path::Vector{String})
      @ FUSE ~/.julia/dev/FUSE/src/data.j

ERROR: LoadError: LoadError: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.
in expression starting at /Users/meneghini/.julia/dev/FUSE/test/runtests_fds.jl:90
in expression starting at /Users/meneghini/.julia/dev/FUSE/test/runtests.jl:4


LoadError: Package FUSE errored during testing

In [7]:
data = FUSE.dd();
resize!(data.core_profiles.profiles_1d,1)
data.core_profiles.profiles_1d[1].grid.rho_tor_norm = Vector{Float64}(collect(1:10))
data.core_profiles.profiles_1d[1].electrons.temperature = Vector{Float64}(collect(1:10))

┌ Error: Failed to revise /Users/meneghini/.julia/dev/FUSE/src/functionarrays.jl
│   exception = Revise.ReviseEvalException("/Users/meneghini/.julia/dev/FUSE/src/functionarrays.jl:318", ErrorException("invalid redefinition of constant NumericalFDVector"), Any[(top-level scope at functionarrays.jl:318, 1)])
└ @ Revise /Users/meneghini/.julia/packages/Revise/AIcio/src/packagedef.jl:707


LoadError: MethodError: no method matching FUSE.NumericalFDVector(::Vector{Vector{Float64}}, ::Vector{Float64})

In [192]:
wall=FUSE.wall()
resize!(wall.description_2d,1)
resize!(wall.description_2d[1].mobile.unit,2)
resize!(wall.description_2d[1].mobile.unit[2].outline,2)

wall__description_2d=FUSE.wall__description_2d()
resize!(wall__description_2d.mobile.unit,2)
resize!(wall__description_2d.mobile.unit[2].outline,2)

"""
    p2i(path::Any[])

Combine list of IMAS location elements into a string
"""
function p2i(path::Vector)
    str = String[]
    for k in path
        if typeof(k) <: Symbol
            push!(str, string(k))
        elseif typeof(k) <: Int
            push!(str, "[$(string(k))]")
        elseif (k == ":") | (k == ':') | (typeof(k) === Colon) 
            push!(str, "[:]")
        elseif typeof(k) <: String
            push!(str, k)
        end
    end
    return replace(join(str, "."), ".[" => "[")
end

"""
    function f2i(fds::Union{FDS,FDSvector})

Return IMAS path
"""
function f2f(fds::Union{FUSE.FDS,FUSE.FDSvector}, child::Union{Missing,FUSE.FDS,FUSE.FDSvector},path::Union{Nothing,Vector}, index::Int)
    if typeof(fds) <: FUSE.FDS
        if typeof(fds._parent.value) <: FUSE.FDSvector
            name = string(Base.typename(typeof(fds)).name)*"___"
        else
            name = string(Base.typename(typeof(fds)).name)
        end
    elseif typeof(fds) <: FUSE.FDSvector
        name = string(Base.typename(eltype(fds)).name)*"___"
    end
    if path === nothing
        path = replace(name, "___"=>"__:__")
        path = Vector{Any}(Vector{String}(split(path, "__")))
        path = [k==":" ? 0 : k for k in path]
        index = length(path)
    end
    if typeof(fds) <: FUSE.FDSvector
        ix = findfirst([k === child for k in fds.value])
        if ix != nothing
            path[index] = ix
        end
    end
    if fds._parent.value === missing
        return p2i(path)
    else
        return f2f(fds._parent.value, fds, path, index-1)
    end
end


println(f2f(wall.description_2d[1].mobile.unit[2].outline[1],missing,nothing,0))
println(f2f(wall__description_2d.mobile.unit[2].outline[1],missing,nothing,0))


Any["wall", "description_2d", 1, "mobile", "unit", 2, "outline", 1, ""]wall.description_2d[1].mobile.unit[2].outline[1].
Any["wall", "description_2d", 0, "mobile", "unit", 2, "outline", 1, ""]wall.description_2d[0].mobile.unit[2].outline[1].


In [82]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
@time data = FUSE.json2fuse(filename; verbose=false)

LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Any}[39m[0m to an object of type [91mFUSE.AbstractFDArray{Float64, 1}[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:58
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractArray at abstractarray.jl:14
[0m  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:205

In [77]:
FDS=FUSE.FDS
FDSvector=FUSE.FDSvector
f2u=FUSE.f2u
imas_info=FUSE.imas_info
top=FUSE.top
i2p=FUSE.i2p
p2i=FUSE.p2i
f2i=FUSE.f2i

"""
    coordinates(fds::FDS, field::Symbol)

Returns two lists, one of coordinate names and the other with their values in the data structure
Coordinate value is `nothing` when the data does not have a coordinate
Coordinate value is `missing` if the coordinate is missing in the data structure
"""
function coordinates(fds::FDS, field::Symbol)
    coord_names = deepcopy(imas_info("$(f2u(fds)).$(field)")["coordinates"])
    coord_values = []
    path = i2p(f2i(fds,stop_at_top))
    for (k, coord) in enumerate(coord_names)
        if contains(coord, "...")
            push!(coord_values, nothing)
        else
            h = top(fds)
            coord = replace(coord, ":" => "1")
            for k in i2p(coord)
                if (typeof(k) <: Int) & (typeof(h) <: FDSvector)
                    if length(keys(h)) <= k
                        h = h[k]
                        #println('*',k)
                    else
                        #println('-',k)
                    end
                elseif (typeof(k) <: String) & (typeof(h) <: FDS)
                    if hasfield(typeof(h), Symbol(k))
                        h = getfield(h, Symbol(k))
                        #println('*',k)
                    else
                        #println('-',k)
                    end
                else
                    error("Something is wrong with the FDS data structure")
                end
            end
            if (h === fds) | (h === missing)
                push!(coord_values, missing)
            else
                push!(coord_values, h)
            end
        end
    end
    return Dict(:names => coord_names, :values => coord_values)
end


#using Test
#@testset "FDS_IMAS" begin
    data = FUSE.dd();
    resize!(data.core_profiles.profiles_1d, 1)

#    # test coordinate of a coordinate
#    coords = FUSE.coordinates(data.core_profiles.profiles_1d[1].grid, :rho_tor_norm)
#    @test coords[:names][1] == "1...N"
#    @test coords[:values][1] === nothing

    # test coordinate of a 1D array (with uninitialized coordinate)
    coords = coordinates(data.core_profiles.profiles_1d[1].electrons, :temperature);
    coords[:names][1];# == "core_profiles.profiles_1d[:].grid.rho_tor_norm"
    #coords[:values][1]# === missing
#end

LoadError: invalid redefinition of constant f2i

In [22]:
using JSON
struct_field_type=FUSE.struct_field_type
FDS=FUSE.FDS
conversion_types=FUSE.conversion_types

"""
    dict2fuse(dct, fds::FDS=dd() ;verbose::Bool=false, path::Vector{String}=String[])::FDS

Populate FUSE data structure `fds` based on data contained in Julia dictionary `dct`.

# Arguments
- `verbose::Bool=false`: print structure hierarchy as it is filled
"""
function dict2fuse(dct, fds::T ;verbose::Bool=false, path::Vector{String}=String[]) where {T <: FDS}
    # recursively traverse `dtc` structure
    level = length(path)
    for (k, v) in dct
        # Struct
        if typeof(v) <: Dict
            if verbose println(("｜"^level) * string(k)) end
            ff = getfield(fds, Symbol(k))
            dict2fuse(v, ff; path=vcat(path, [string(k)]), verbose=verbose)

        # Array of struct
        elseif (typeof(v) <: Array) && (length(v) > 0) && (typeof(v[1]) <: Dict)
            ff = getfield(fds, Symbol(k))
            if verbose println(("｜"^level) * string(k)) end
            resize!(ff, length(v))
            for i in 1:length(v)
                if verbose println(("｜"^(level + 1)) * string(i)) end
                dict2fuse(v[i], ff[i]; path=vcat(path, [string(k),"[$i]"]), verbose=verbose)
            end

        # Leaf
        else
            if verbose print(("｜"^level) * string(k) * " → ") end
            target_type = typeintersect(conversion_types, struct_field_type(typeof(fds), Symbol(k)))
            if target_type <: AbstractArray
                if ndims(target_type) == 2
                    v = reduce(hcat, v)
                end
                v = convert(Array{eltype(target_type),ndims(target_type)}, v)
            end
            setproperty!(fds, Symbol(k), v)
            if verbose println(typeof(v)) end
        end
    end

    return fds
end

"""
    json2fuse(filename::String; verbose::Bool=false)::FDS

Load from a file with give `filename` the OMAS data structure saved in JSON format 

# Arguments
- `verbose::Bool=false`: print structure hierarchy as it is filled
"""
function json2fuse(filename::String; verbose::Bool=false)::FDS
    fds_data = FUSE.dd()
    json_data = JSON.parsefile(filename)
    dict2fuse(json_data, fds_data; verbose=verbose)
    return fds_data
end

filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
data  = json2fuse(filename; verbose=false)


EQUILIBRIUM
｜time_slice
｜｜[1]
｜｜｜time ➡ Float64
｜｜｜profiles_1d
｜｜｜｜b_field_max ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜dvolume_drho_tor ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜gm9 ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜dpsi_drho_tor ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜surface ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜rho_tor ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜b_field_min ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜darea_dpsi ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜squareness_upper_inner ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜squareness_lower_inner ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜rho_tor_norm ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜elongation ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜b_field_average ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜psi ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜gm8 ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜dpressure_dpsi ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜triangularity_upper ➡ 17-element FUSE.NumericalFDVector
｜｜｜｜darea_drho_tor ➡ 17-element FUSE.Numer

In [8]:
v=Array{Float64}
p=Array{}

Array{Float64, N} where N